In [8]:
import pandas as pd
import statsmodels.api as sm

In [5]:
wege_gemeinden_geo = pd.read_csv(
    "generated_data/wege_gemeinden_geo.csv", sep=";", encoding="utf-8"
)

In [7]:
wege_gemeinden_geo.columns

Index(['HHNR', 'Wegnummer', 'Distanz_km', 'Emissionen', 'Verkehrsmittel',
       'Etappennummer', 'Verkehrsmittel_first', 'Zweck', 'Distanz_km_first',
       'Start_X', 'Start_Y', 'Ziel_X', 'Ziel_Y', 'Freizeitaktivität_x',
       'gwp_pro_Kilometer_[g]', 'Emissionen_first', 'WEGNR',
       'Start_X-Koordinate_x', 'Start_Y-Koordinate_x', 'Ziel_X-Koordinate_x',
       'Ziel_Y-Koordinate_x', 'Freizeitaktivität', 'Dauer in Minuten',
       'Wegzweck', 'Alter', 'Geschlecht', 'Führerausweis PW',
       'Führerausweis Motorräder', 'Lernfahrausweis PW',
       'Lernfahrausweis Motorräder', 'Ausbildungsniveau', 'Beschäftigungsgrad',
       'Wetter', 'Velo-Besitzer', 'Mofa-Besitzer', 'Kleinmotorrad-Besitzer',
       'Motorrad-Besitzer', 'Auto-Besitzer', 'Familienstand', 'Erwerbsstatus',
       'Tägliche Freizeitwege', 'Wohnort X-Koordinate', 'Wohnort Y-Koordinate',
       'W_ORT', 'Wohnort: PLZ', 'Wohnort: Kanton', 'Führerausweis_PW',
       'Führerausweis_Motorrad', 'Start_X-Koordinate_y',
    

In [9]:
# Auswahl der relevanten Spalten
features = ["Alter", "Typologie", "Ausbildungsniveau"]
target = "Emissionen"

In [10]:
dummies = pd.get_dummies(
    wege_gemeinden_geo[features],
    drop_first=False,  # Alle Kategorien beibehalten
)

In [ ]:
# Festlegen der Referenzkategorien
dummies = dummies.drop(
    ["Alter_81-100", "Typologie_Grosszentren", "Ausbildungsniveau_Keine Schulbildung"],
    axis=1,
)

In [13]:
# Kombinieren der Dummy-Variablen mit der Zielvariable
data = pd.concat([dummies, wege_gemeinden_geo[target]], axis=1)

# Features (X) und Zielvariable (y) definieren
X = data.drop(columns=target)
y = data[target]

# Hinzufügen eines Interzepts zur linearen Regression
X = sm.add_constant(X)

# Erstellen des Modells
model = sm.OLS(y, X)

# Fitting des Modells
results = model.fit()

# Zusammenfassung des Modells anzeigen
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:             Emissionen   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     5.090
Date:                Tue, 31 Dec 2024   Prob (F-statistic):           6.17e-13
Time:                        17:29:28   Log-Likelihood:            -6.0259e+05
No. Observations:               55751   AIC:                         1.205e+06
Df Residuals:                   55730   BIC:                         1.205e+06
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                                                                                  coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------